In [1]:
import json
import pandas as pd
from time import time
from katm_connections import get_mongo_client
from katm_utils import insert_into_mssql, load_my_columns, \
                        get_numbers,map_dff_to_my_columns, max_number_find, \
                        select

In [2]:
start = time()
# get_nums_table='bronze.katm_077_contingent_liabilities'
# foreign_key = 'contingent_liabilities_qty'
write_to_table = 'bronze.katm_077_contingent_liabilities_offline'
columns_file = 'katm_077_contingent_liabilities_columns.txt'
columns = load_my_columns(columns_file)

In [3]:
client = get_mongo_client()
db = client['katm']
task_collection = db['katm_cache']

query = {
    'reports.077.contingent_liabilities.contingent_liability': {'$exists': True}
    # ,'number': {'$eq': 5651095}
    # ,'number': {'$gt': max_num}
    # ,'number': {'$gt': 1243100, '$lt': 1243200}
    # ,'number': {'$in': numbers}   
}
projection = {
    '_id': 1,
    'reports.077.contingent_liabilities.contingent_liability': 1
}

In [4]:
docs = task_collection.find_one(query, projection)

In [5]:
docs

{'_id': ObjectId('65616d529967f3feb5c98968'),
 'reports': {'077': {'contingent_liabilities': {'contingent_liability': {'max_uninter_overdue_percent': '',
     'overdue_debt_sum': '',
     'contract_closing_date': '',
     'contract_id': '',
     'contract_status': '',
     'max_current_uninter_overdue_percent': '',
     'branch': '',
     'amount_change': '',
     'percent': '',
     'contract_end_date': '',
     'contract_date': '',
     'total_debt_sum': '',
     'currency': '',
     'max_current_overdue_change': '',
     'org_name': '',
     'percent_change': '',
     'max_overdue_principal_change': '',
     'amount': '',
     'subject_type': '',
     'org': '',
     'overdue_debt_sum_change': '',
     'max_overdue_principal': '',
     'max_current_overdue': '',
     'max_current_uninter_overdue_percent_change': '',
     'max_uninter_overdue_percent_change': '',
     'total_debt_sum_change': '',
     'loan_sibject': ''}}}}}

In [6]:
max_date = select('select max(cast(contract_date as date)) from bronze.katm_077_contracts_offline')[0].strftime('%Y-%m-%d')

In [7]:
max_date

'2024-02-26'

In [8]:
client = get_mongo_client()
db = client['katm']
task_collection = db['katm_cache']

query = {
    'reports.077.contracts.contract': {'$exists': True}
    # ,'number': {'$eq': 8141290}
    # ,'number': {'$gt': max_num}
    # ,'number': {'$gt': 1243100, '$lt': 1243200}
    # ,'number': {'$in': numbers}
    , 'reports.077.contracts.contract.contract_date':{'$gte':max_date}
}
projection = {
    '_id': 1,
    'reports.077.contracts.contract': 1
}
docs = task_collection.find_one(query, projection)

In [9]:
docs

{'_id': ObjectId('65dc153cd3270e2feaf70117'),
 'reports': {'077': {'contracts': {'contract': [{'reserve_bal_change': '',
      'contract_id': '617909',
      'security_qty': '1',
      'contract_status': '1',
      'revised_principal_sum': '',
      'offbalance_princial_sum_change': '',
      'branch': '01183',
      'securities': {'security': {'pinfl': '-',
        'amount': '832000000',
        'subject_type': '',
        'contract_id': '617909',
        'inn': '',
        'security_change': '2024-02-23',
        'contract_date': '',
        'juridical_status': '2',
        'name': "ABILXASI'MOV SULTANBEK YUSUP-ULI'",
        'currency': '000',
        'consent_date': '22.02.24',
        'security_type': '73',
        'consent_id': '617909'}},
      'class_asset_quality': '1',
      'forecasted_schedule': {'forecasted_schedule_change': '2024-02-26',
       'forecasted_schedule_change_qty': '1',
       'forecasted_payment': [{'forecasted_payment_period': '2024-02',
         'principal

In [12]:
import json
import pandas as pd
from time import time
from katm_connections import get_mongo_client
from katm_utils import insert_into_mssql, load_my_columns, \
                        map_dff_to_my_columns

In [13]:
start = time()
write_to_table = 'bronze.asbt_list_online_microloan'
columns_file = 'asbt_credit_list_online_microloan.txt'
columns = load_my_columns(columns_file)

In [14]:
client = get_mongo_client()
db = client['task']
task_collection = db['task']

query = {
    'data.asbt_credit_list.return.object': {'$exists': True}
}
projection = {
    'number': 1,
    'data.asbt_credit_list.return.object': 1
}

In [15]:
docs = task_collection.find_one(query, projection)

In [16]:
docs

{'_id': ObjectId('63ef578b59db4840e879875a'),
 'number': 79,
 'data': {'asbt_credit_list': {'return': {'object': [{'idMfo': '937',
      'id': '93700014295813',
      'globId': '93700002157049',
      'nikiId': '0002157049',
      'numDog': '13+1',
      'dateDog': '2023-02-14 00:00:00.0',
      'client': 'MUXITDINOVA NARGIZA ABDULAZIZOVNA',
      'namAcc': 'Ипотека ва истеъмол кредитлари учун (ФИО)',
      'idClient': '4893193',
      'accNew': '14901000662038997801',
      'status': '0',
      'inspect': '1180',
      'kodVal': '0',
      'kodSost': '3',
      'procSs': '28',
      'procPr': '42',
      'basPer': '365',
      'dateVydD': '2023-02-14 00:00:00.0',
      'datePogD': '2025-02-13 00:00:00.0',
      'sumRas': '100000000',
      'mejBank': '0',
      'summa': '100000000',
      'sumObesp': '130000000',
      'srok': '3',
      'vidCred': '32',
      'celSsudy': '60024',
      'typeOpesp': '51',
      'istCred': '11',
      'numTran': '801',
      'charVal': 'UZS',
      'na

In [17]:
rows=[]

number = docs.get('number')
fields = docs.get('data', {}).get('asbt_credit_list', {}).get('return', {}).get('object', [])
if isinstance(fields, dict):
    fields = [fields]
elif fields is None:
    fields = []
for field in fields:
    # filtered_field = {k: v for k, v in field.items() if not isinstance(v, (dict, list))}
    row = {'number': number, **field}
    rows.append(row)

In [18]:
df = pd.DataFrame(rows)

In [19]:
df

,number,idMfo,id,globId,nikiId,numDog,dateDog,client,namAcc,idClient,...,repaymentAccountRest,summaGashP,summaPr,summaPrOsn,summaPrProc,summaGNext,dateGash,summaGMin,saldoIn,saldoF
0,79,937,93700014295813,93700002157049,0002157049,13+1,2023-02-14 00:00:00.0,MUXITDINOVA NARGIZA ABDULAZIZOVNA,Ипотека ва истеъмол кредитлари учун (ФИО),4893193,...,0,230136,0,0,0,5488848,2023-03-03 00:00:00.0,5488848,100000000,100230136
1,79,937,93700014300760,93700002157486,0002157486,13+1,2023-02-15 00:00:00.0,MUXITDINOVA NARGIZA ABDULAZIZOVNA,Ипотека ва истеъмол кредитлари учун (ФИО),4893193,...,0,153424,0,0,0,5488848,2023-03-03 00:00:00.0,5488848,100000000,100153424


In [20]:
df.columns

Index(['number', 'idMfo', 'id', 'globId', 'nikiId', 'numDog', 'dateDog',
       'client', 'namAcc', 'idClient', 'accNew', 'status', 'inspect', 'kodVal',
       'kodSost', 'procSs', 'procPr', 'basPer', 'dateVydD', 'datePogD',
       'sumRas', 'mejBank', 'summa', 'sumObesp', 'srok', 'vidCred', 'celSsudy',
       'typeOpesp', 'istCred', 'numTran', 'charVal', 'namStatus', 'nameIns',
       'nameType', 'saldo', 'repaymentAccount', 'repaymentAccountRest',
       'summaGashP', 'summaPr', 'summaPrOsn', 'summaPrProc', 'summaGNext',
       'dateGash', 'summaGMin', 'saldoIn', 'saldoF'],
      dtype='object')

In [1]:
from katm_utils import max_number_find_with_cast

In [3]:
max_num = max_number_find_with_cast('bronze.asbt_list_online_microloan')

In [4]:
max_num

970220

In [6]:
from katm_connections import get_mongo_client

In [7]:
client = get_mongo_client()
db = client['task']
task_collection = db['task']

query = {
    'data.asbt_credit_list.return.object': {'$exists': True}
    ,'number': {'$gt': max_num}
}
projection = {
    'number': 1,
    'data.asbt_credit_list.return.object': 1
}

In [8]:
doc = task_collection.find_one(query,projection)

In [9]:
doc

{'_id': ObjectId('655d82b68c5d13926a656719'),
 'number': 970221,
 'data': {'asbt_credit_list': {'return': {'object': [{'idMfo': '122',
      'id': '12200016519976',
      'globId': '12200002236328',
      'nikiId': '0002236328',
      'numDog': '140678/2023',
      'dateDog': '2023-05-25 00:00:00.0',
      'client': 'XOLBOYEV BUNYOD NORBOY O‘G‘LI',
      'namAcc': 'Ипотека ва истеъмол кредитлари учун (XOLBOYEV BUNYOD NORBOY O‘G‘LI)',
      'idClient': '7802961',
      'accNew': '14901000463508792001',
      'status': '0',
      'inspect': '8860',
      'kodVal': '0',
      'kodSost': '3',
      'procSs': '24',
      'procPr': '28.8',
      'basPer': '365',
      'dateVydD': '2023-05-25 00:00:00.0',
      'datePogD': '2026-05-22 00:00:00.0',
      'sumRas': '4670000000',
      'mejBank': '0',
      'summa': '4670000000',
      'sumObesp': '5837500000',
      'srok': '3',
      'vidCred': '32',
      'celSsudy': '60024',
      'typeOpesp': '41',
      'istCred': '11',
      'numTran': '8

In [10]:
from datetime import datetime

In [11]:
print(datetime(2023,11,1))

2023-11-01 00:00:00


In [12]:
import json
import pandas as pd
from time import time
from datetime import datetime
from katm_connections import get_mongo_client
from katm_utils import insert_into_mssql_tqdm, load_my_columns, \
                        map_dff_to_my_columns, max_number_find_with_cast


In [13]:
client = get_mongo_client()
db = client['task']
task_collection = db['task']

date_filter = datetime(2023,11,1)

In [16]:
date_filter

datetime.datetime(2023, 11, 1, 0, 0)

In [75]:
query = {
    'data.katm_077.result': {'$exists': True},
    # 'number': {'$eq': 9447946},
    # 'created': {'$gt': date_filter},
    'data.katm_077.resutl.is_ok': 'false'
}

projection = {
    'number': 1,
    'created': 1,
    'request.clientId': 1,
    'data.katm_077.result': 1
}


In [76]:
doc = task_collection.find_one(query, projection)

In [74]:
doc

{'_id': ObjectId('67c697b9d63e2511c45a1c8c'),
 'request': {'clientId': '4271524'},
 'created': datetime.datetime(2025, 3, 4, 6, 3, 37, 74000),
 'number': 9447946,
 'data': {'katm_077': {'result': {'is_ok': True,
    'data': {'expired_summa': 0,
     'is_exists_stop_factor': False,
     'summa_other_mikrozaym': 5840742.92,
     'summa_other_avtocredit': 83327189.73,
     'summa_other_ipoteka': 0,
     'today_payment': 4776690,
     'loan_summa': 91254493,
     'num_all_credits_ib': 3,
     'num_all_other': 5,
     'num_all_IB': 3,
     'bureau_segment': 'ib',
     'num_active_micro_ib': 1,
     'sum_active_micro_ib': 5840742.92,
     'scoring_level': 'ХОРОШИЙ',
     'scoring_class': 'B1',
     'scoring_grade': 322,
     'score_cutoff': 180,
     'current_dlq_bureau': 0,
     'current_dlq_total': 0}}}}}

In [56]:
number = doc.get('number')

In [57]:
number

9447946

In [58]:
clientId = doc.get('request').get('clientId')

In [59]:
clientId

'4271524'

In [63]:
is_ok = doc.get('data', {}).get('katm_077', {}).get('result', {}).get('is_ok')

In [64]:
is_ok

True

In [65]:
rows=[]
fields = doc.get('data', {}).get('katm_077', {}).get('result', {}).get('data',{})
if isinstance(fields, dict):
    fields = [fields]
elif fields is None:
    fields = []
for field in fields:
    # filtered_field = {k: v for k, v in field.items() if not isinstance(v, (dict, list))}
    row = {'number': number, 'clientId': clientId, 'is_ok':is_ok, **field}
    rows.append(row)

In [66]:
df = pd.DataFrame(rows)

In [67]:
df

,number,clientId,is_ok,expired_summa,is_exists_stop_factor,summa_other_mikrozaym,summa_other_avtocredit,summa_other_ipoteka,today_payment,loan_summa,...,num_all_IB,bureau_segment,num_active_micro_ib,sum_active_micro_ib,scoring_level,scoring_class,scoring_grade,score_cutoff,current_dlq_bureau,current_dlq_total
0,9447946,4271524,True,0,False,5840742.92,83327189.73,0,4776690,91254493,...,3,ib,1,5840742.92,ХОРОШИЙ,B1,322,180,0,0


In [72]:
query = {
    'data.katm_077.result': {'$exists': True}
    # ,'number': {'$gt': max_num}
    ,"created": {"$gt": date_filter}
    # ,"created":{"$lt": datetime(2023,12,15)}
    ,"katm_077.result.is_ok": {"$in": [False, "false"]}
}
projection = {
    'number': 1,
    'created': 1,
    'request.clientId': 1,
    'katm_077.result.is_ok':1,
    'data.katm_077.result.error': 1
}

In [73]:
doc = task_collection.find_one(query, projection)

KeyboardInterrupt: 

In [77]:
import json
import pandas as pd
from time import time, sleep
from datetime import datetime
from pymongo.errors import CursorNotFound
from katm_connections import get_mongo_client
from katm_utils import insert_into_mssql_tqdm

In [78]:
write_to_table = 'bronze.hints'
date_filter = datetime(2023, 11, 1)
query = {
    'data.katm_077.result': {'$exists': True}
    ,"created": {"$gt": date_filter}
}
projection = {
    'number': 1,
    'data.katm_077.result': 1
}

In [1]:
import json
import pandas as pd
from time import time
from datetime import datetime
from katm_connections import get_mongo_client
from katm_utils import insert_into_mssql, load_my_columns, \
                        get_numbers,map_dff_to_my_columns, max_number_find, \
                        select


In [2]:
client = get_mongo_client()
db = client['katm']
task_collection = db['katm_cache']

In [9]:
query = {
    'reports.077.open_contracts.open_contract': {'$exists': True}
    ,'time': {'$gt': datetime(2024, 1, 1)} ,
    # ,'number': {'$eq': 8141290}
    # ,'number': {'$gt': max_num}
    # ,'number': {'$gt': 1243100, '$lt': 1243200}
    # ,'number': {'$in': numbers}
    # , 'reports.077.contracts.contract.contract_date':{'$gte':max_date}
}
projection = {
    '_id': 1,
    'time':1,
    'reports.077.contracts.contract': 1
}

In [10]:
docs = task_collection.find_one(query, projection)

In [11]:
docs

{'_id': ObjectId('6592df71b4f16a115baf7190'),
 'time': datetime.datetime(2024, 1, 1, 15, 51, 13, 298000),
 'reports': {'077': {'contracts': {'contract': [{'reserve_bal_change': '2022-09-09',
      'contract_id': '1933571',
      'security_qty': '1',
      'contract_status': '1',
      'revised_principal_sum': '',
      'offbalance_princial_sum_change': '',
      'branch': '00544',
      'securities': {'security': {'pinfl': '',
        'amount': '2210000000',
        'subject_type': '2',
        'contract_id': '27-092 030',
        'inn': '306 317 501',
        'security_change': '2022-07-07',
        'contract_date': '',
        'juridical_status': '1',
        'name': 'ООО  IMKON SUGURTA',
        'currency': '000',
        'consent_date': '07.07.22',
        'security_type': '51',
        'consent_id': '27-092/030'}},
      'class_asset_quality': '1',
      'forecasted_schedule': {'forecasted_schedule_change': '2022-07-08',
       'forecasted_schedule_change_qty': '1',
       'foreca

In [6]:
import json

In [12]:
json_tree = {"output": list(docs)}
with open("output.json", "w", encoding="utf-8") as json_file:
    json.dump(docs, json_file, indent=4, default=str, ensure_ascii=False)


In [13]:
import json
import pandas as pd
from time import time
from datetime import datetime
from katm_connections import get_mongo_client
from katm_utils import insert_into_mssql_tqdm, load_my_columns, \
                        get_numbers,map_dff_to_my_columns, max_number_find, \
                        select

In [14]:
client = get_mongo_client()
db = client['katm']
task_collection = db['katm_cache']

In [15]:
query = {
    'reports.077.overview': {'$exists': True}
    ,'time': {'$gt': datetime(2024, 1, 1)} ,
    # ,'number': {'$eq': 8141290}
    # ,'number': {'$gt': max_num}
    # ,'number': {'$gt': 1243100, '$lt': 1243200}
    # ,'number': {'$in': numbers}
    # , 'reports.077.contracts.contract.contract_date':{'$gte':max_date}
}
projection = {
    '_id': 1,
    'reports.077.overview': 1
}

In [16]:
docs = task_collection.find_one(query, projection)

In [17]:
docs

{'_id': ObjectId('6592df71b4f16a115baf7190'),
 'reports': {'077': {'overview': {'contingent_liabilities_qty': '5',
    'max_overdue_principal_sum': '136714123',
    'max_uninter_overdue_percent_days': '438',
    'max_overdue_principal_days': '197',
    'total_overdue_percent_sum': '2564882293',
    'credit_request_qty': '53',
    'claims_qty': '50',
    'average_monthly_payment': '306042323',
    'subscriptions_qty': '0',
    'contracts_qty': '6',
    'overdue_principal_qty': '110',
    'actual_average_monthly_payment': '98117365.99'}}}}

In [26]:
import pandas as pd
from katm_connections import get_mongo_client
from katm_utils import insert_into_mssql_tqdm

In [27]:
write_to_table = 'bronze.katm_077_cache_ids'
client = get_mongo_client()
db = client['crm']
collection = db['action']

In [28]:
query = {
    'credit.katm_cache_id': {'$exists': True}
}
projection = {
    'number': 1,
    'credit.katm_cache_id': 1
}

In [29]:
docs = collection.find_one(query, projection)

In [30]:
docs

{'_id': ObjectId('5ebcd880878a2268f5e8af30'),
 'credit': {'katm_cache_id': ObjectId('5ebe3db5e3560207c0877d56')},
 'number': '4/2020'}

In [33]:
rows = []

In [31]:
number = docs.get('number')
katm_cache_id = docs.get('credit').get('katm_cache_id')

In [32]:
row = {'number': number, 'katm_cache_id': katm_cache_id}

In [34]:
rows.append(row)

In [35]:
df = pd.DataFrame(rows)

In [36]:
df

,number,katm_cache_id
0,4/2020,5ebe3db5e3560207c0877d56


In [38]:
table_name = 'bronze.katm_077_cache_ids'

In [39]:
check_table_query = f"""
    IF OBJECT_ID(N'{table_name}', 'U') IS NULL
    BEGIN
        CREATE TABLE {table_name}  (
    {', '.join([f'[{col}] NVARCHAR(1000)' for col in df.columns])}
);
    END;"""

In [40]:
from katm_connections import get_sql_server_connection

In [41]:
conn = get_sql_server_connection()

In [42]:
cursor = conn.cursor()

In [43]:
cursor.execute(check_table_query)

In [44]:
conn.commit()

In [1]:
import json
import pandas as pd
from time import time
from datetime import datetime
from katm_connections import get_mongo_client
from katm_utils import insert_into_mssql_tqdm, load_my_columns, \
                        get_numbers,map_dff_to_my_columns, max_number_find, \
                        select

In [4]:
client = get_mongo_client()
db = client['katm']
collection = db['katm_cache']

In [5]:
query = {
    'reports.077.contracts.contract.overdue_principals.overdue_principal': {'$exists': True}
}
projection = {
    '_id': 1,
    'reports.077.contracts.contract.overdue_principals.overdue_principal': 1
}

In [6]:
doc = collection.find_one(query,projection)

In [7]:
doc

{'_id': ObjectId('65616fa9d74ab7d4fa160ed4'),
 'reports': {'077': {'contracts': {'contract': {'overdue_principals': {'overdue_principal': [{'overdue_principal_sum': '102492400',
        'overdue_date': '2022-09-06',
        'overdue_principal_change': '2022-10-02',
        'overdue_principal_days': '24'},
       {'overdue_principal_sum': '107696408',
        'overdue_date': '2022-10-05',
        'overdue_principal_change': '2022-10-19',
        'overdue_principal_days': '13'},
       {'overdue_principal_sum': '16525450',
        'overdue_date': '2022-12-05',
        'overdue_principal_change': '2022-12-16',
        'overdue_principal_days': '10'},
       {'overdue_principal_sum': '8787967',
        'overdue_date': '2023-03-03',
        'overdue_principal_change': '2023-03-07',
        'overdue_principal_days': '3'},
       {'overdue_principal_sum': '43015845',
        'overdue_date': '2023-04-05',
        'overdue_principal_change': '2023-04-08',
        'overdue_principal_days': '2'},

In [8]:
query = {
    'reports.077.contracts.contract': {
        '$elemMatch': {
            'overdue_principals.overdue_principal': {'$exists': True}
        }
    }
}
projection = {
    '_id': 1,
    'reports.077.contracts.contract': 1
}

In [11]:
docs = collection.find_one(query, projection)

In [12]:
docs

{'_id': ObjectId('65617e4bd74ab7d4fa16106a'),
 'reports': {'077': {'contracts': {'contract': [{'reserve_bal_change': '2023-11-09',
      'contract_id': '2212678',
      'security_qty': '1',
      'contract_status': '1',
      'revised_principal_sum': '',
      'offbalance_princial_sum_change': '',
      'branch': '00223',
      'securities': {'security': {'pinfl': '31401 7429 40030',
        'amount': '5000000000',
        'subject_type': '3',
        'contract_id': '1061',
        'inn': '',
        'security_change': '2023-05-02',
        'contract_date': '',
        'juridical_status': '2',
        'name': 'AXMEDOV MUXTARALI ASKAROVICH',
        'currency': '000',
        'consent_date': '28.04.23',
        'security_type': '41',
        'consent_id': '1061'}},
      'class_asset_quality': '1',
      'forecasted_schedule': {'forecasted_schedule_change': '2023-05-02',
       'forecasted_schedule_change_qty': '1',
       'forecasted_payment': [{'forecasted_payment_period': '2023-05',


In [27]:
from bson import ObjectId

In [64]:
query = {
    'reports.077.contracts.contract.overdue_procents.overdue_procent': {'$exists': True}
  }
projection = {
    '_id': 1,
    'reports.077.contracts.contract': 1
}

In [65]:
doc = collection.find_one(query,projection)

In [50]:
doc

{'_id': ObjectId('67d278a933d7494b8fd0d780'),
 'reports': {'077': {'contracts': {'contract': [{'overdue_principals': {'overdue_principal': [{'overdue_principal_sum': '215791714',
         'overdue_date': '2023-10-02',
         'overdue_principal_change': '2024-02-06',
         'overdue_principal_days': '118'},
        {'overdue_principal_sum': '108000000',
         'overdue_date': '2023-11-02',
         'overdue_principal_change': '2024-03-05',
         'overdue_principal_days': '120'},
        {'overdue_principal_sum': '357892847',
         'overdue_date': '2023-12-02',
         'overdue_principal_change': '2025-03-13',
         'overdue_principal_days': '467'}]}},
     {'overdue_principals': {'overdue_principal': [{'overdue_principal_sum': '34899416',
         'overdue_date': '2023-09-07',
         'overdue_principal_change': '2023-10-03',
         'overdue_principal_days': '22'},
        {'overdue_principal_sum': '35655570',
         'overdue_date': '2023-10-06',
         'overdue_p

In [60]:
docs = collection.find(query, projection)

In [59]:
print(collection.count_documents(query))

357402


In [61]:
rows = []
batch_size = 10
for i, doc in enumerate(docs, 1):
    number = doc.get('number')
    _id = doc.get('_id')
    print(_id)
    fields = doc.get('reports', {}).get('077', {}).get('contracts', {}).get('contract', [])
    if isinstance(fields, dict):
        fields = [fields]
    for contract in fields:
        contract_id = contract.get('contract_id')
        overdue_principals = contract.get('overdue_principals')
        if isinstance(overdue_principals, dict):
            overdue_principal = overdue_principals.get('overdue_principal')
            if isinstance(overdue_principal, dict):
                overdue_principal = [overdue_principal]
            elif overdue_principal is None:
                overdue_principal = []
            for i in range(len(overdue_principal)):
                row = {"_id":_id, 'contract_id': contract_id, 'number': number, **overdue_principal[i]}
                rows.append(row)
    if i % batch_size == 0:
        break

65616fa9d74ab7d4fa160ed4
65617e4bd74ab7d4fa16106a


In [62]:
rows

[{'_id': ObjectId('65616fa9d74ab7d4fa160ed4'),
  'contract_id': None,
  'number': None,
  'overdue_principal_sum': '102492400',
  'overdue_date': '2022-09-06',
  'overdue_principal_change': '2022-10-02',
  'overdue_principal_days': '24'},
 {'_id': ObjectId('65616fa9d74ab7d4fa160ed4'),
  'contract_id': None,
  'number': None,
  'overdue_principal_sum': '107696408',
  'overdue_date': '2022-10-05',
  'overdue_principal_change': '2022-10-19',
  'overdue_principal_days': '13'},
 {'_id': ObjectId('65616fa9d74ab7d4fa160ed4'),
  'contract_id': None,
  'number': None,
  'overdue_principal_sum': '16525450',
  'overdue_date': '2022-12-05',
  'overdue_principal_change': '2022-12-16',
  'overdue_principal_days': '10'},
 {'_id': ObjectId('65616fa9d74ab7d4fa160ed4'),
  'contract_id': None,
  'number': None,
  'overdue_principal_sum': '8787967',
  'overdue_date': '2023-03-03',
  'overdue_principal_change': '2023-03-07',
  'overdue_principal_days': '3'},
 {'_id': ObjectId('65616fa9d74ab7d4fa160ed4'),
 

In [63]:
print(len(rows))

79


In [56]:
print(collection.count_documents(query))

380736


In [1]:
import json
import pandas as pd
from time import time
from datetime import datetime
from katm_connections import get_mongo_client
from katm_utils import  insert_into_mssql_tqdm,\
                        get_numbers, map_dff_to_my_columns, \
                        max_number_find, load_my_columns

In [2]:
start = time()
write_to_table = 'bronze.employer'
get_nums_table = 'bronze.employer'
columns_file = 'employer_fields.txt'
columns = load_my_columns(columns_file)
# max_num = max_number_find(get_nums_table)
client = get_mongo_client()
db = client['task']
task_collection = db['task']

In [3]:
query = {
    'data.uzasbo_salary.result.data.lastEmployer': {'$exists': True}
    # ,'number':{"$gt": max_num}
    ,'created': {'$gt': datetime(2025, 1, 1)}
}
projection = {
    '_id':1,
    'number': 1,
    'data.uzasbo_salary.result.data.lastEmployer': 1
}

In [4]:
docs = task_collection.find(query, projection).limit(10)

In [14]:
doc

{'_id': ObjectId('63f612e7caa9f44927b732f8'),
 'number': 444,
 'data': {'uzasbo_salary': {'result': {'data': {'lastEmployer': {'inn': '200541366',
      'title': 'Мирзо Улугбек тумани аҳоли бандлигига кўмаклашиш маркази',
      'type': 1,
      'regionID': 2}}}}}}

In [5]:
i = 0
rows=[]
for doc in docs:
    # if i% 10_000 == 0:
    #     df = pd.DataFrame(rows)
    #     final_df = map_dff_to_my_columns(df, columns)
    #     insert_into_mssql_tqdm(final_df, write_to_table)
    #     rows = []    
    
    # i+=1
    _id = str(doc.get('_id')) 
    number = doc.get('number')
    fields = doc.get('data', {}).get('uzasbo_salary', {}).get('result',{}).get('data', {}).get('lastEmployer', {})
    if isinstance(fields, dict):
        fields = [fields]
    elif fields is None:
        fields = []
    for field in fields:
        # filtered_field = {k: v for k, v in field.items() if not isinstance(v, (dict, list))}
        row = {'_id': _id, 'number': number, **field}
        rows.append(row)
        
df = pd.DataFrame(rows)
final_df = map_dff_to_my_columns(df, columns)
insert_into_mssql_tqdm(df, write_to_table)
end = time()
print(f"script took {end-start} seconds")

Inserting rows: 100%|██████████| 10/10 [00:00<00:00, 746.09row/s]

script took 11.306510925292969 seconds


In [25]:
df

,_id,number,inn,title,type,districtOrg
0,6774886771463695f8a096bb,8352839,202311548,Кашкадарё вилоят ички ишлар бошкармаси,10,43
1,67749127c4739799b764c66b,8352857,309219709,Ёшлар ишлари агентлиги Ўзбекистон тумани булими,30,163
2,6774916c71463695f8a09783,8352858,202238147,102-сон Заиф эшитувчи болалар махсус мактаб ин...,26,204
3,677491e9c4739799b764c67f,8352859,203317482,54-сон умумий урта таълим мактаби,18,96
4,677492ee71463695f8a097a5,8352864,201311152,58-умумий урта таълим мактаби,14,68
5,67749554c4739799b764c6de,8352871,203353470,Околтин тумани 8-сонли умумтаълим мактаби,24,118
6,677495f471463695f8a097f3,8352872,202255902,Карши Маданият ва Туризм КХК,10,43
7,677498d271463695f8a09829,8352878,201876827,17-умумий урта таълим мактаби,30,150
8,67749a68c4739799b764c761,8352885,200905687,Кушкупир тумани Тиббиёт бирлашмаси,33,176
9,67749a88c4739799b764c76e,8352887,200056794,Наманган шахар 6-оила поликлиникаси,14,68


In [ ]:
# Function to create an empty DataFrame with specified columns
def create_empty_df_with_columns(columns):
    return pd.DataFrame(columns=columns)

# Function to load columns from a text file
def load_my_columns(file_path):
    with open(file_path, 'r') as file:
        columns = file.read().splitlines()  # Read each line as a column name
    return columns

# Function to map `dff` values to a DataFrame with specified columns
def map_dff_to_my_columns(dff, my_columns):
    # Ensure '_id' and 'number' are in the my_columns list
    essential_columns = ['_id', 'number']
    for col in essential_columns:
        if col not in my_columns:
            my_columns.append(col)

    # Create an empty DataFrame with the columns
    final_df = create_empty_df_with_columns(my_columns)

    # Fill in the columns of `final_df` with values from `dff`
    for col in final_df.columns:
        if col in dff.columns:
            final_df[col] = dff[col]  # Copy the values from `dff`
        else:
            final_df[col] = None  # Set the column to None if not in `dff`

    return final_df

In [27]:
columns

'employer_fields.txt'